<a href="https://colab.research.google.com/github/kh-abd-kh/Figures/blob/main/Calculating_Heegner_Points_in_Sage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sage code for calculating Heegner points
#
# This code calculates Heegner points on elliptic curves over Q.
#
# Input:
#     E: An elliptic curve defined over Q (using Sage's EllipticCurve).
#     D: A negative fundamental discriminant.
#
# Output:
#     A Heegner point on E corresponding to the discriminant D,
#     or None if the conditions are not met.
#
# Algorithm:
# 1. Check that the discriminant D is a negative fundamental discriminant.
# 2. Check that the Hilbert class polynomial for D has an integer root modulo N,
#    where N is the level of the elliptic curve.
# 3. Compute a root r of the Hilbert class polynomial modulo N.
# 4. Compute a complex root tau of the Hilbert class polynomial that
#    is in the upper half-plane and satisfies the condition that
#    (a*tau + b)/(c*tau + d) is in the same fundamental domain as tau,
#     where a,b,c,d are the coefficients of a matrix in SL2(Z).
# 5. Compute the elliptic curve over C with complex multiplication by
#    the order of discriminant D.
# 6. Compute the Weierstrass equation of the elliptic curve over C.
# 7. Compute the isomorphism from C/Lattice to the elliptic curve over C.
# 8. Evaluate the Weierstrass equation at the point corresponding to tau.
# 9. Map the point to E using the modular parametrization.
#
# Note:
# - This code uses the modular parametrization of the elliptic curve.
# - The Heegner point is defined over the Hilbert class field of Q(sqrt(D)).
# - The algorithm involves complex numbers and numerical approximations.
# - The accuracy of the result depends on the precision used in the computations.

def heegner_point(E, D):
    """
    Calculates a Heegner point on the elliptic curve E corresponding to the
    negative fundamental discriminant D.

    Args:
        E: An elliptic curve defined over Q (using Sage's EllipticCurve).
        D: A negative fundamental discriminant (an integer).

    Returns:
        A point on E (an element of E(Qbar)), or None if the conditions are not met.

    Raises:
        TypeError: If E is not an elliptic curve or D is not an integer.
        ValueError: If D is not a negative fundamental discriminant,
                    or if the conditions for the existence of a Heegner point
                    are not satisfied.
    """
    if not isinstance(E, EllipticCurve):
        raise TypeError("E must be an elliptic curve.")
    if not isinstance(D, int):
        raise TypeError("D must be an integer.")

    if D >= 0:
        raise ValueError("D must be negative.")

    if not is_fundamental_discriminant(D):
        raise ValueError("D must be a fundamental discriminant.")

    N = E.conductor()
    if not kronecker(D, N) == 1:
        print(f"Kronecker symbol (D/N) = {kronecker(D, N)} != 1.  No Heegner point.")
        return None

    # Calculate the Hilbert class polynomial
    K = QuadraticField(D)
    try:
        H = hilbert_class_polynomial(K)
    except:
        print("Failed to compute Hilbert class polynomial.")
        return None

    # Check for a root modulo N
    Hp = H.change_ring(IntegerModRing(N))
    if not Hp.roots():
        print(f"Hilbert class polynomial has no root modulo N = {N}. No Heegner point.")
        return None
    r = Hp.roots()[0][0]  # Take the first root

    # Compute a complex root of the Hilbert class polynomial
    # We need to find a root in the upper half-plane.
    # We start by finding any complex root.
    complex_roots = H.roots(ring=CC)
    if not complex_roots:
        print("Failed to find complex roots of the Hilbert class polynomial.")
        return None

    tau = None
    for root, mult in complex_roots:
        if root.imag() > 0:
            tau = root
            break
    if tau is None:
        print("No complex root in the upper half-plane found.")
        return None

    # Ensure tau is in the fundamental domain.  This is tricky, and
    # the following is NOT a complete solution, but it's a start.  A
    # proper implementation requires careful handling of SL2(Z)
    # transformations.  This part is often the source of errors.
    if tau.real() > 0.5:
        tau = tau - 1
    elif tau.real() < -0.5:
        tau = tau + 1
    if abs(tau) < 1:
        tau = -1/tau

    # j-invariant of the elliptic curve with CM by the order of discriminant D
    j_val = H(tau) # Should be very close to an integer.

    # Compute the elliptic curve over C.
    try:
        EC = EllipticCurve(j=j_val)
    except:
        print(f"Failed to create elliptic curve from j-invariant {j_val}.")
        return None
    # Get the period lattice
    omega1, omega2 = EC.period_lattice_basis()
    L = omega1 * ZZ + omega2 * ZZ

    # Compute the Weierstrass p-function and its derivative.
    wp = EC.weierstrass_p()
    dwp = EC.weierstrass_p_prime()

    # Evaluate the Weierstrass function at tau.
    P_complex = (wp(tau), dwp(tau))

    # Map the complex point to E using the modular parametrization.
    # This is the crucial step where we use the modular parametrization.
    # Sage provides this functionality through the parameterization() method.

    phi = E.modular_parametrization()
    try:
        P = phi(tau)
    except:
        print("Error mapping to E using modular parametrization.  tau =", tau)
        return None
    return P

def is_fundamental_discriminant(D):
    """
    Checks if D is a fundamental discriminant.

    Args:
        D: An integer.

    Returns:
        True if D is a fundamental discriminant, False otherwise.
    """
    if D == 1:
        return True
    if D % 4 == 1:
        return is_squarefree(D)
    elif D % 4 == 0:
        return is_squarefree(D // 4) and (D // 4) % 4 in [2, 3]
    else:
        return False

def hilbert_class_polynomial(K):
    """
    Computes the Hilbert class polynomial of the quadratic field K.

    Args:
        K: A quadratic field (using Sage's QuadraticField).

    Returns:
        The Hilbert class polynomial (a univariate polynomial in Sage).
    """
    return K.hilbert_class_polynomial()

def kronecker(D, n):
    """
    Computes the Kronecker symbol (D/n).

    Args:
        D: An integer.
        n: A non-negative integer.

    Returns:
        The Kronecker symbol (D/n).
    """
    return kronecker_symbol(D, n)

# Example usage:
# E = EllipticCurve('37a')  # An elliptic curve over Q
# D = -43                 # A negative fundamental discriminant
# P = heegner_point(E, D)
# if P:
#     print(f"Heegner point for D = {D} on E: {P}")
# else:
#     print(f"Could not compute Heegner point for D = {D} on E.")

# Example 2
# E = EllipticCurve([0,0,1,-1,0]) # y^2 + xy = x^3 - x^2
# D = -23
# P = heegner_point(E, D)
# if P is not None:
#   print(P)